In [26]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from nltk.corpus import stopwords
from scipy.sparse import hstack
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import re
from sklearn.decomposition import TruncatedSVD
from utils1 import *
import string
from pattern.en import suggest
import snowballstemmer
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))

In [27]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r') as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [28]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

def text_process(text):
    
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed = tokenizer.tokenize(text)
    text_processed = ' '.join(word for word in text_processed if word not in STOPWORDS)
#     porter_stemmer = PorterStemmer()
#     text_processed = [porter_stemmer.stem(word) for word in text_processed]
    return text_processed

In [29]:
def clean_df(tw):
    tw["Tweet"].replace(r'http.?://[^\s]+[\s]?','', regex=True,inplace=True)
    tw['Tweet'] = tw['Tweet'].str.lower()
    tw["Tweet"].replace(r"@\S+", " ", regex=True ,inplace=True)
#     tw["Tweet"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
#     tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
#     tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
#     tw["Tweet"].replace(r"\d+", "NUM", regex=True,inplace=True)
    tw["Tweet"].replace('[^a-zA-Z\s]', '', regex=True,inplace=True)
    tw['num_of_words'] = tw["Tweet"].str.split().apply(len)
    tw.drop(tw[tw.num_of_words<3].index, inplace=True)
    return tw

In [30]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
tw = clean_df(tw)
tw['Tweet'] = tw["Tweet"].apply(text_process)

In [31]:
def rename_user(user):
    user = f'__label__{user}'
    return user

In [32]:
tw['User'] = tw['User'].apply(rename_user)

In [33]:
tw.sample(10,random_state = 0)

,User,Tweet,num_of_words
134123,__label__9949,definately want,4
209106,__label__4925,truthout revealed war fraud whistleblowers wra...,11
117889,__label__4893,flues dont even fireplace,7
175905,__label__5729,haha looking buried booty ill leave another fa...,15
120782,__label__3494,really love coffee glad whoever invented,13
223958,__label__7832,red ring killer youre screwed man thats got ri...,14
159409,__label__2828,start premiership today already good games exc...,17
315423,__label__2226,ever get feeling life comedy act world audience,17
179499,__label__67,sf aka favorite american city place im destine...,14
185080,__label__337,hmm inspire craft tofunicorn,8


In [34]:
X_train, X_test, y_train, y_test = train_test_split(tw.Tweet,tw.User, random_state=0,test_size = .3)

In [35]:
def write_file(X,y,filename):
    with open(filename, 'w') as writeFile:
        for user,tweet in zip(y,X):
            writeFile.write(f'{user} {tweet}\n')
    

In [36]:
write_file(X_train,y_train,'trainData.train')
write_file(X_test,y_test,'testData.train')

In [37]:
import fasttext

In [ ]:
model = fasttext.train_supervised(
    input="trainData.train", 
    lr=0.2, 
    epoch=25, 
    wordNgrams=2, 
    bucket=200000, 
    dim=100, 
    loss='ova')